In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from skimage.io import imread

from keras.applications import resnet50
from keras.optimizers import Adam
from keras.layers import GlobalAveragePooling2D, Dense
from keras.models import Model
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
train = pd.read_csv('../input/aerial-cactus-identification/train.csv')
test = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')
y_train = np.array(train.has_cactus)

In [3]:
X_train = []
for name in train.id:
    X_train.append(imread('../input/aerial-cactus-identification/train/train/'+name))

In [4]:
X_test = []
for name in test.id:
    X_test.append(imread('../input/aerial-cactus-identification/test/test/'+name))

In [5]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [11]:
#Preprocessing the inputs
X_train = resnet50.preprocess_input(X_train)
X_test = resnet50.preprocess_input(X_test)

In [12]:
#Retriving the model without the fully connected layers
base_model = resnet50.ResNet50(include_top=False, weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.


/opt/conda/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step


In [13]:
#Let's add a GAP and a fully connected layer with relu function
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [14]:
model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0001), metrics=['accuracy'])

In [16]:
#If have never trained this CNN, you can comment this line
model.load_weights('../input/weights/model.hdf5')

In [17]:
#Let's overfit in one batch
model.fit(X_train[:128], y_train[:128], epochs=30)

Instructions for updating:
Use tf.cast instead.
Epoch 1/30
128/128 [==============================] - 13s 102ms/step - loss: 0.0166 - acc: 0.9922
Epoch 2/30
128/128 [==============================] - 0s 2ms/step - loss: 0.0042 - acc: 1.0000
Epoch 3/30
128/128 [==============================] - 0s 2ms/step - loss: 3.0094e-04 - acc: 1.0000
Epoch 4/30
128/128 [==============================] - 0s 2ms/step - loss: 6.1113e-04 - acc: 1.0000
Epoch 5/30
128/128 [==============================] - 0s 2ms/step - loss: 1.0666e-04 - acc: 1.0000
Epoch 6/30
128/128 [==============================] - 0s 2ms/step - loss: 2.6171e-05 - acc: 1.0000
Epoch 7/30
128/128 [==============================] - 0s 2ms/step - loss: 2.1797e-05 - acc: 1.0000
Epoch 8/30
128/128 [==============================] - 0s 2ms/step - loss: 2.7657e-05 - acc: 1.0000
Epoch 9/30
128/128 [==============================] - 0s 2ms/step - loss: 1.8799e-04 - acc: 1.0000
Epoch 10/30
128/128 [==============================] - 0s 2ms/step

In [18]:
#Now we can train the model and save the weights in a .hdf5 file
checkpoint = ModelCheckpoint(filepath='model.hdf5')
model.fit(X_train, y_train, epochs=100, batch_size=128, callbacks=[checkpoint])

Epoch 1/100
17500/17500 [==============================] - 13s 730us/step - loss: 0.0035 - acc: 0.9990


KeyboardInterrupt: 

In [19]:
sample = pd.read_csv('../input/aerial-cactus-identification/sample_submission.csv')

In [ ]:
sample.has_cactus = model.predict(X_test)

In [ ]:
#Let's plot the distribution of the test targets
fig, ax = plt.subplots()
sns.kdeplot(sample.has_cactus, ax=ax)
ax.set_xlim([0,1])

In [20]:
sample.to_csv('sample_submission.csv', index=False)